In [2]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score


# Configuration
CONFIG = {
    "IMAGES_DIR_TRAINING": "./Training",
    "IMAGES_DIR_TESTING": "./Testing",
    "CLASSES": ["glioma", "meningioma", "notumor", "pituitary"],
    "TRAIN_RATIO": 0.9,
    "VALIDATE_RATIO": 0.1,
    "BATCH_SIZE": 8,
    "IMAGE_SIZE": 224,
    "IMAGE_DIVISION": 255.0,
    "N_EPOCHS": 30,
    "LEARNING_RATE": 1e-5,
    "NUM_CLASSES": 4,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Num GPUs Available:", torch.cuda.device_count())

# Data transformations
transform = transforms.Compose([
    transforms.Resize((CONFIG["IMAGE_SIZE"], CONFIG["IMAGE_SIZE"])),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

# Load dataset
train_dataset = ImageFolder(CONFIG["IMAGES_DIR_TRAINING"], transform=transform)
train_size = int(CONFIG["TRAIN_RATIO"] * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, num_workers=0, pin_memory=False)
val_loader = DataLoader(val_dataset, batch_size=CONFIG["BATCH_SIZE"], shuffle=False, num_workers=0, pin_memory=False)

# Define Model
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 256),
    nn.ReLU(),
    nn.Linear(256, CONFIG["NUM_CLASSES"]),
    nn.Softmax(dim=1)
)
model = model.to(device)

# Freeze layers except last 50
for param in list(model.features.parameters())[:-50]:
    param.requires_grad = False

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=CONFIG["LEARNING_RATE"])

# Training Loop with Validation Metrics
for epoch in range(CONFIG["N_EPOCHS"]):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training Phase
    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_val += predicted.eq(labels).sum().item()
            total_val += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = 100. * correct_val / total_val

    # Compute Precision, Recall, and F1-score
    precision = precision_score(all_labels, all_preds, average="weighted")
    recall = recall_score(all_labels, all_preds, average="weighted")
    f1 = f1_score(all_labels, all_preds, average="weighted")

    print(f"Epoch {epoch+1}/{CONFIG['N_EPOCHS']} - "
          f"Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}% - "
          f"Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}% - "
          f"Precision: {precision:.4f} - Recall: {recall:.4f} - F1-score: {f1:.4f}")


# Compute class weights
def count_images_per_class(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts

class_counts = count_images_per_class(CONFIG["IMAGES_DIR_TRAINING"])
class_labels = CONFIG["CLASSES"]
class_indices = {class_name: i for i, class_name in enumerate(class_labels)}

class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(class_labels)),
    y=np.concatenate([np.full(count, class_indices[class_name]) for class_name, count in class_counts.items()])
)

class_weight_dict = {i: weight for i, weight in enumerate(class_weight)}
print("Class Weights:", class_weight_dict)


Num GPUs Available: 1
Epoch 1/30 - Train Loss: 1.3537 - Train Acc: 59.34% - Val Loss: 1.3067 - Val Acc: 72.03% - Precision: 0.7602 - Recall: 0.7203 - F1-score: 0.7070
Epoch 2/30 - Train Loss: 1.2442 - Train Acc: 66.71% - Val Loss: 1.1543 - Val Acc: 76.92% - Precision: 0.7746 - Recall: 0.7692 - F1-score: 0.7484
Epoch 3/30 - Train Loss: 1.1105 - Train Acc: 74.14% - Val Loss: 1.0224 - Val Acc: 81.99% - Precision: 0.8185 - Recall: 0.8199 - F1-score: 0.8093
Epoch 4/30 - Train Loss: 1.0300 - Train Acc: 78.81% - Val Loss: 0.9720 - Val Acc: 86.71% - Precision: 0.8670 - Recall: 0.8671 - F1-score: 0.8657
Epoch 5/30 - Train Loss: 0.9858 - Train Acc: 82.10% - Val Loss: 0.9275 - Val Acc: 87.59% - Precision: 0.8754 - Recall: 0.8759 - F1-score: 0.8743
Epoch 6/30 - Train Loss: 0.9602 - Train Acc: 82.74% - Val Loss: 0.9100 - Val Acc: 87.06% - Precision: 0.8713 - Recall: 0.8706 - F1-score: 0.8696
Epoch 7/30 - Train Loss: 0.9411 - Train Acc: 84.09% - Val Loss: 0.8837 - Val Acc: 88.99% - Precision: 0.8938

In [3]:
torch.save(model, "brain_tumor_model.pth")